In [1]:
from mip import *
from itertools import product
from parse import *
import time
import numpy as np

Using Python-MIP package version 1.7.2


In [2]:
# ,,,,,,,,,,,,,, Обнуление тех значений, между которыми нет каналов ,,,,,,,,,,,,,,

def zero_nconnect(model, q, N, V, M, Bandwidth):
    for (k, i, j) in product(N, V, V):
        if Bandwidth.at[i, j] == 0:
            for t in M:
                model += q[i][j][k][t] == 0

In [3]:
# ,,,,,,,,,,,,,, Запрещаем учитывать время после t_k для каждого контракта ,,,,,,,

def only_selftime(model, q, K, V, N, T, e):
    for k in N:
        for i, j in product(V, V):
            for t in range(K[k]['time'] // e, T // e):
                model += q[i][j][k][t] == 0

In [4]:
# ,,,,,,,,,,,,,, Условие 0 ,,,,,,,,,,,,,,

def eval_cnstr0(min_t, M, V, K, N, Bandwidth, BackFlows):
    for t in M:
        sum_v = 0
        for i, j in product(V, V):
            if Bandwidth.at[i, j] != 0:
                sum_v += (int)(Bandwidth.at[i, j] - BackFlows.at[i, j])
        if not (sum_v >= sum(K[k]['volume'] for k in N) / min_t):
            return False
    return True

In [5]:
# ,,,,,,,,,,,,,, Условие 1 ,,,,,,,,,,,,,,

def eval_cnstr1(model, q, M, V, N, Bandwidth, BackFlows):
    for t in M:
        for i in V:
            for j in V:
                if Bandwidth.at[i, j] != 0:
                    model += xsum(q[i][j][k][t] for k in N) <= (int)(Bandwidth.at[i, j] - BackFlows.at[i, j])


In [6]:
# ,,,,,,,,,,,,,, Условие 2 ,,,,,,,,,,,,,,

def eval_cnstr2(model, q, K, N, V, e):
    for k in N:
        a_k = K[k]['A']
        t_k = K[k]['time']
        model += xsum(q[a_k][j][k][t] * e for t in range(t_k // e) for j in V) == K[k]['volume']

In [7]:
# ,,,,,,,,,,,,,, Условие 3 ,,,,,,,,,,,,,,

def eval_cnstr3(model, q, K, N, V, e):
    for k in N:
        a_k = K[k]['A']
        t_k = K[k]['time']
        model += xsum(q[j][a_k][k][t] * e for t in range(t_k // e) for j in V) == 0

In [8]:
# ,,,,,,,,,,,,,, Условие 4 ,,,,,,,,,,,,,,

def eval_cnstr4(model, q, K, N, V, e):
    for k in N:
        b_k = K[k]['B']
        t_k = K[k]['time']
        model += xsum(q[j][b_k][k][t] * e for t in range(t_k // e) for j in V) == K[k]['volume']

In [9]:
# ,,,,,,,,,,,,,, Условие 5 ,,,,,,,,,,,,,,

def eval_cnstr5(model, q, K, N, V, e):
    for k in N:
        b_k = K[k]['B']
        t_k = K[k]['time']
        model += xsum(q[b_k][j][k][t] * e for t in range(t_k // e) for j in V) == 0

In [10]:
# ,,,,,,,,,,,,,, Условие 6 ,,,,,,,,,,,,,,

def eval_cnstr6(model, q, M, N, K, V, Bandwidth):
    for t in M:
        for k in N:
            a_k = K[k]['A']
            b_k = K[k]['B']
            for r in V - {a_k, b_k}:
                sum_in = 0
                sum_out = 0
                flag = False
                for (j1, j2) in product(V, V):
                    if Bandwidth.at[j2, r] != 0:
                        sum_in += q[j2][r][k][t]
                        flag = True
                    if Bandwidth.at[r, j1] != 0:
                        sum_out += q[r][j1][k][t]
                        flag = True
                if flag:
                    model += ((sum_in - sum_out) == 0)

In [11]:
# ,,,,,,,,,,,,,, Условие 7 ,,,,,,,,,,,,,,

def eval_cnstr7(model, q, M, V, N):
    for t in M:
        for k in N:
            for i in V:
                for j in V:
                    model += q[i][j][k][t] >= 0

In [12]:
# ,,,,,,,,,,,,,, Условие 8 ,,,,,,,,,,,,,,

def eval_cnstr8(model, q, N, K, e, V):
    for i, j in product(V, V):
        for k in N:
            t_k = K[k]['time']
            for t in range(t_k // e - 1):
                model += (q[i][j][k][t] == q[i][j][k][t + 1])

In [13]:
def get_result(status, model, V, K, M, time):
    correctness = ""
    answer = np.zeros([len(V), len(V), len(K), len(M)])
    if (status != None) and (status == OptimizationStatus.OPTIMAL):
        correctness = "correct"
        for i in range(len(model.vars)):
            pattern = 'q[{:d}][{:d}][{:d}][{:d}]'
            i1, i2, i3, i4 = parse(pattern, model.vars[i].name)
            answer[i1][i2][i3][i4] = model.vars[i].x
    else:
        correctness = "Model infeasible"
        
    return (correctness, answer, time)

In [14]:
def evaluate_model(V, N, M, K, T, e, Bandwidth, BackFlows):
    
    start_time = time.time()

    model = Model()

    q = [[[[model.add_var(name = 'q[{}][{}][{}][{}]'.format(i,j,k,t),
              var_type = INTEGER)  for t in range(T // e)] 
                                   for k in N] 
                                   for j in V] 
                                   for i in V]

    min_t = T;
    for k in N:
        min_t = min(K[k]['time'], min_t)

    zero_nconnect(model, q, N, V, M, Bandwidth)
    only_selftime(model, q, K, V, N, T, e)
    if not eval_cnstr0(min_t, M, V, K, N, Bandwidth, BackFlows):
        end_time = time.time()
        return get_result(None, model, V, K, M, end_time - start_time)
        
    eval_cnstr1(model, q, M, V, N, Bandwidth, BackFlows)
    eval_cnstr2(model, q, K, N, V, e)
    eval_cnstr3(model, q, K, N, V, e)
    eval_cnstr4(model, q, K, N, V, e)
    eval_cnstr5(model, q, K, N, V, e)
    eval_cnstr6(model, q, M, N, K, V, Bandwidth)
    eval_cnstr7(model, q, M, V, N)
    eval_cnstr8(model, q, N, K, e, V)
    
    model.objective = minimize(xsum(q[i][j][k][t] for i, j, k, t in product(V, V, N, M)))

    status = model.optimize()
    
    end_time = time.time()
    
    return get_result(status, model, V, K, M, end_time - start_time)